# **HOPOMO IMPLEMENTATION**




### **IMPORTS**

In [1361]:
 !pip install astral

In [1362]:
from functools import lru_cache
from plotly.subplots import make_subplots
from plotly import tools
from itertools import islice
from astral import *
from astral.sun import *
from datetime import *
from pprint import pprint

import requests
import json
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import plotly.graph_objs as go


### **PARAMETRES**



In [1363]:
# Comportements de la reine/ponte
x1, x2, x3, x4, x5 = 385, 30, 36, 155, 30 #REF p230 # x4 = jour de l'année où la reine pond le plus
ELRbase = 1600 #REF p230 - taux de ponte moyen de la reine (dépend de la reine, son âge etc ..)
SUPtreshold = 0.2 #REF p230 # si le taux de cellules vides est inférieur à cette limite alors il y a une diminution du taux de ponte (la reine a de plus en plus de mal à trouver des cellules disponibles)
ELRstochrange = 0 # ?** # facteur stochastique du taux de ponte de la reine

# Cellules
CELLShive = 250000 #REF p230 # nb de cellules dans la ruche

# Initialisations
BEESadultINIT = 15000
STOREShoneyINIT = 50000

# Couvain
LIFESPANegg = 3 #REF p230 #durée max (en jours) de l'état oeuf
LIFESPANlarvae = 5 #REF p230 #durée max (en jours) de l'état larve
LIFESPANpupae = 12 #REF p230 #durée max (en jours) de l'état pupe 
MORTALITYeggs = 0.03 #REF p230 # Mortalités (Plus le stade est avancé et moins le taux de mortalité est élevé)
MORTALITYlarvae = 0.01 #REF p230
MORTALITYpupae = 0.001 #REF p230
CANNIBALISMbase = [0.23,0.30,0.58,0.06,0] #REF p230 # Taux fixe de cannibalisme des larves en fonction de leur age (1-5 jours)

# Abeilles adultes
MORTALITYadultbase = 0.01 #REF p230
MORTALITYnursing = 0.005 #REF p230
MORTALITYprocessing = 0.005 #REF p230
MORTALITYforaging = 0.035 #REF p230

# Facteurs liés aux ressources
FACTORpollensavingmax = 0.30 #REF p230
RATIOnectar_to_honey = 0.4 #REF p230 20/50
FACTORothertasks = 0.2 # ??******************
FACTORpollenstorage = 6 #REF p230 - means that the colony regulates the pollen stores around a level that represents a reserve for approximately 6 days, based on the current level of demand
FACTORminpollenforagers = 0.01 #REF p230
FACTORforagingmax=0.33 #REF p230

# Besoins
NECTARNEEDactiveforager = 0.03 # 
NECTARNEEDnurse = 0.02 #?****
NECTARNEEDadult = 0.005 #?**** PAS DEFINI mais important
NECTARNEEDlarva = [0.0025,0.0050,0.0084,0.016,0.032] #REF p231 graph b
POLLENNEEDlarva = [0.0150,0.0250,0.0400,0.080,0.156] #REF p231 graph b
POLLENNEEDadult = 0.0037 # inspiré de Rortais et al. (2005) article
POLLENNEEDnurse = 0.045 # inspiré de Rortais et al. (2005) article
NEEDnurses_per_larva = [0.10,0.30,0.70,1.70,3.0] #REF p231 graph d # à déterminer, augmentation exponentielle grandissant avec la masse de la larve
NEEDnurses_per_egg = 0.30 # à établir, moins important que pour les larves car pas besoin d'être nourrit
NEEDnurses_per_pupa = 0.30 # à établir, moins important que pour les larves car pas besoin d'être nourrit

#Butinage
LOADpollenforager = 0.06 # cellfulls REF p230
LOADnectarforager = 0.04 # cellfulls REF p230
FACTORforagingsuccess = 0.8 #REF p230
TURNSnectarforager = 15 #REF p230
TURNSpollenforager = 10 #REF p230
ProcessorsPerCell = 2  #REF p230
stochasticForagingFactor = 0.25

# Poids (en grammes)
w_hivebase = 14000   #REF p231
w_cellsbase = 0.037  #REF p231 #represents the weight of one empty cell (wax only).
w_pollen = 0.23  #REF p231 # poids d'une cellule remplie de pollen
w_nectar = 0.43  #REF p231 # poids d'une cellule remplie de nectar
w_honey = 0.50  #REF p231 #poids d'une cellule remplie de miel
w_egg = 0.0001  #REF p231
w_larva = [0.0002,0.00059,0.00331,0.0644,0.160] #le poids d'une larve à l'état i ∈ {1,2,3,4,5}
w_pupa = 0.16  #REF p231
w_adult = 0.10  #REF p231

max_wind = 45 #km/h 
min_humidity = 60 #pourcentage

### **METEOROLOGIE ET TRAITEMENTS**

**PREDICTIONS**

Cette API permet d'obtenir une prédiction du temps par toutes les 3h sous format JSON jusqu'à 5 jours. Le code ci-dessous recupère ces prédictions ainsi que les données météos actuelles à Pau.

In [1364]:
# API Key générée sur le site
api_key = '2a7f4f5c2a0ae6e3fce7b81c64a61956'
lat = "43.3" #Latitude de Pau
lon = "-0.3667" #Longitude de Pau
city_name = "Pau"
url = "https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&appid=%s&units=metric" % (lat, lon, api_key)

forecast_url = "http://api.openweathermap.org/data/2.5/forecast?q=Pau,fr&appid=2a7f4f5c2a0ae6e3fce7b81c64a61956"
 
weather_data_forecast = requests.get(forecast_url).json()

print("\nForecast Weather Data Of " + city_name +":\n")
pprint(weather_data_forecast)


Forecast Weather Data Of Pau:

{'city': {'coord': {'lat': 43.4167, 'lon': -0.5833},
          'country': 'FR',
          'id': 2988356,
          'name': 'Arrondissement de Pau',
          'population': 300287,
          'sunrise': 1613890303,
          'sunset': 1613929230,
          'timezone': 3600},
 'cnt': 40,
 'cod': '200',
 'list': [{'clouds': {'all': 50},
           'dt': 1613919600,
           'dt_txt': '2021-02-21 15:00:00',
           'main': {'feels_like': 287.59,
                    'grnd_level': 992,
                    'humidity': 56,
                    'pressure': 1006,
                    'sea_level': 1006,
                    'temp': 290.49,
                    'temp_kf': -1.36,
                    'temp_max': 291.85,
                    'temp_min': 290.49},
           'pop': 0,
           'sys': {'pod': 'd'},
           'visibility': 10000,
           'weather': [{'description': 'scattered clouds',
                        'icon': '03d',
                        'id'

In [1365]:
#Import des données des CSV
data2020 = pd.read_csv("/content/meteo_gelos_owm_2020.csv",skiprows=[i for i in range(1394,1407)],sep =',') #exclusion 29/02 pour simplifier les processus 

#Données globales
dataMeteo2020 = pd.DataFrame(data2020,columns=['date','main.temp','rain.1h','main.humidity','wind.speed'])

#Conversion en listes des données météo
list_DataMeteo2020 = [list(row) for row in dataMeteo2020.values]


In [1366]:
#Traitement des listes pour le modèle

#Conversion de la date et de l'heure puis transformation de ces deux variables
#en des listes.

def convert_date(listeIn) :
  for i in range(len(listeIn)):
    listeIn[i][0] = listeIn[i][0].split('T')
    listeIn[i][0] = listeIn[i][0][0].split('-') + listeIn[i][0][1].split('.')
    del listeIn[i][0][4]
    listeIn[i][0][3] = listeIn[i][0][3].split(':')
    listeIn[i][0] = listeIn[i][0][0:3] + listeIn[i][0][3]
    
    for j in range(6):
      listeIn[i][0][j] = int(listeIn[i][0][j])

    listeIn[i].insert(0,listeIn[i][0][0:3])
    listeIn[i].insert(1,listeIn[i][1][3:])
    del listeIn[i][2]
  
  return "Traitement réalisé avec succès"

convert_date(list_DataMeteo2020)

'Traitement réalisé avec succès'

In [1367]:
# Utilisation de Astral pour obtenir : l'heure du levé de soleil, du coucher
# Ainsi que le nombre d'heures d'ensoleillement.
# A partir de ces données nous pourront exclures les données non pertinentes
# (heures de non ensoleillement) des data de "list_DataMeteo2020"

# Il existe néanmoins des méthodes plus complexes et plus précises
# de l'ordre de la minute pour calculer le nombre effectif d'heures 
# d'ensoleillement (cf angle terrestre etc...)
# La méthode utilisée dans Astral a été adaptée à Python.
# https://www.esrl.noaa.gov/gmd/grad/solcalc/calcdetails.html 

city = LocationInfo('Pau', 'France', 'Europe/London', 43.2951, -0.370797)

sunhours = []
mois_longs = [1,3,5,7,8,10,12]

# Réupère les horaires sur un an (en 2020) des couchés et levés de soleil
# La sortie est une liste de 365 elements de 2 listes
# (une liste pour le lever et une autre pour le coucher)
def getSunSetRise():
  for i in range(1,13):
    if (i in mois_longs) :
      for j in range(1,32):
        elem = daylight(city.observer,date=dt.date(2020,i,j),tzinfo=city.timezone)
        sunhours.append(convert_dataSun(elem))
    elif (i == 2) :
      for j in range(1,29):
        elem = daylight(city.observer,date=dt.date(2020,i,j),tzinfo=city.timezone)
        sunhours.append(convert_dataSun(elem))
    elif ((i not in mois_longs) and i!=2):
      for j in range(1,31):
        elem = daylight(city.observer,date=dt.date(2020,i,j),tzinfo=city.timezone)
        convert_dataSun(elem)
        sunhours.append(convert_dataSun(elem))
  return "Récupérations des data effectuée avec succès"


#Transforme la sortie d'astral (datetime.datetime) en liste d'int
def convert_dataSun(listeIn):
  res = []
  for i in range(len(listeIn)):
      tmp = str(listeIn[i])
      tmp = tmp.split('.')
      tmp = tmp[0].split(' ')
      tmp[0] = tmp[0].split('-')
      tmp[1] = tmp[1].split(':')
      res.append(tmp[0])
      res.append(tmp[1])
      for j in range(len(res)):
        for k in range(len(res[j])):
          res[j][k] = int(res[j][k])
  del res[2]
  return res

In [1368]:
getSunSetRise()

'Récupérations des data effectuée avec succès'

In [1369]:
#Convertit le jour (int) en une date [yy,mm,dd] (list)
def date_converter(daynumber):
  year = str(list_DataMeteo2020[0][0][0])
  day_num = str(daynumber)
  if (day_num == '0' or day_num== '-1'):
    res = [0,0,0]
    return res
  else :
    res = datetime.strptime(year + "-" +day_num, "%Y-%j").strftime("%m-%d-%Y")
    res = res.split('-')
    for i in range(len(res)):
      res[i] = int(res[i])
    res.reverse()
    res[1],res[2] = res[2],res[1]
    return res

#Récupère les data du jour en fonction du jour t
def get_dataOfTheDay(t):
  date_formated = date_converter(t)
  sun_rise = sunhours[t][1]
  sun_set = sunhours[t][2]
  resultat = []

  for i in range(len(list_DataMeteo2020)):
    if ((list_DataMeteo2020[i][0][1] == date_formated[1]) and (list_DataMeteo2020[i][0][2] == date_formated[2])):
      if ((sun_rise[0] <= list_DataMeteo2020[i][1][0]) and (sun_set[0] >= list_DataMeteo2020[i][1][0])):
        resultat.append(list_DataMeteo2020[i])
  return resultat

In [1370]:
len(sunhours)

365

### **FACTEUR SAISON (2.1)**


In [1371]:
#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def f(t):
    return(1-1/(1+x1*np.exp(-2*t/x2)))

#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def g(t):
    return(1/(1+x3*np.exp(-2*(t-x4)/x5))) 

#--------------------------------------------------------------#  

#Facteur saison (t en jours [max 365 jours])
@lru_cache(maxsize = None)
def season(t):
    return max(g(t),f(t))

#--------------------------------------------------------------#

In [1372]:
#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def f(t):
    return(1-1/(1+x1*np.exp(-2*t/x2)))

#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def g(t):
    return(1/(1+x3*np.exp(-2*(t-x4)/x5))) 

#--------------------------------------------------------------#  

#Facteur saison (t en jours [max 365 jours])
@lru_cache(maxsize = None)
def season(t):
    return max(g(t),f(t))

#--------------------------------------------------------------#

### **CELLULES DANS LA RUCHE ET TAUX DE PONTE (2.1)**

In [1373]:

#--------------------------------------------------------------#

# Nombre de cellules disponibles (vides)
@lru_cache(maxsize = None)
def CELLSempty(t) :
    if (t<=0):
        return CELLShive
    else :
        return int(CELLShive - CELLSbrood(t)-STORESpollen(t)-STORESnectar(t)-STOREShoney(t))

#--------------------------------------------------------------#

# Fonction décrivant la diminution du taux de ponte lorsque le nombre de cellules disponibles devient limité 
@lru_cache(maxsize = None)
def SUPcomb(t):
    res = CELLSempty(t)/(CELLShive+1)
    if (res < SUPtreshold):
        return res / SUPtreshold
    else :
        return 1  

#--------------------------------------------------------------#

# Facteur stochastique (aléatoire) du taux de ponte
@lru_cache(maxsize = None)
def ELRstoch():
    return rd.uniform(-ELRstochrange,ELRstochrange)

#--------------------------------------------------------------#

# Fonction prédictive du taux de ponte de la reine
@lru_cache(maxsize = None) 
def ELR(t):
    return ELRbase*(1+ELRstoch())*(1-season(t))*SUPcomb(t)  

#--------------------------------------------------------------#    

### **OEUFS DANS LA RUCHE (2.2)**

In [1374]:
#--------------------------------------------------------------#    

# Nombre d'oeuf au stade i à un instant t
@lru_cache(maxsize = None)
def EGGS(i,t):
    if (t<=0) :
        return 0
    # i désigne l'âge de l'individu immature, t le jour
    if i==1 :
        return ELR(t-1) * (1-MORTALITYeggs)
    elif i==2 or i==3 :
        return EGGS(i-1,t-1)*(1-MORTALITYeggs)
    else :
        # Exception, la fonction EGGS n'est pas définie pour i qui n'appartient pas à [|1;3|]
        print('Erreur, l age i n est pas possible pour la fonction EGGS daprès le modele')
        return (-1)

#--------------------------------------------------------------#    

# Nombre de cellules contenant des oeufs dans la ruche à un instant t
@lru_cache(maxsize = None) 
def CELLSeggs(t):
  # Nombre total d'oeuf dans la ruche à un instant t 
  return int(sum([EGGS(i,t) for i in range(1,LIFESPANegg+1)]))

#--------------------------------------------------------------#   

### **LARVES DANS LA RUCHE (2.2)**

In [1375]:
#--------------------------------------------------------------#

# Taux de cannibalisme des larves
@lru_cache(maxsize = None)
def CANNIBALISMlarvae(i,t):
    return(CANNIBALISMbase[i-1]*(1-(INDEXpollensituation(t-1)*INDEXnursingquality(t-1))))

#--------------------------------------------------------------#  

# Taux de larves survivantes
@lru_cache(maxsize = None)
def SURVIVALlarvae(i,t):
    return((1-CANNIBALISMlarvae(i,t))*(1-MORTALITYlarvae))

#--------------------------------------------------------------#  

# Nombre de larves au stade i à un instant t
@lru_cache(maxsize = None)
def LARVAE(i,t):
    if (t<=0) :
        return 0
  # i désigne l'âge de l'individu immature, t le jour considéré
    if i==1 :
        return EGGS(LIFESPANegg,t-1)*SURVIVALlarvae(1,t)
    elif i>1 and i<=LIFESPANlarvae :
        return LARVAE(i-1,t-1)*SURVIVALlarvae(i,t)
    else :
        # Exception, la fonction LARVAE n'est pas définie pour i qui n'appartient pas à [|1;LIFESPANlarvae|]
        print('Erreur, l age i n est pas possible pour la fonction LARVAE daprès le modele')
        return (-1)

#--------------------------------------------------------------#

# Nombre de cellules contenant des larves dans la ruche à un instant t
@lru_cache(maxsize = None) 
def CELLSlarvae(t):
  # Nombre total de larves dans la ruche à un instant t  
  return int(sum([LARVAE(i,t) for i in range(1,LIFESPANlarvae+1)]))

#--------------------------------------------------------------#

### **PUPES DANS LA RUCHE (2.2)**

In [1376]:
#--------------------------------------------------------------#

# Nombre de pupes au stade i à un instant t
@lru_cache(maxsize = None)
def PUPAE(i,t):
    if (t<=0) :
        return 0
    # i désigne l'âge de l'individu immature, t le jour
    if i==1 :
        return int(LARVAE(LIFESPANlarvae,t-1)*(1-MORTALITYpupae))
    elif i>1 and i<=LIFESPANpupae :
        return int(PUPAE(i-1,t-1)*(1-MORTALITYpupae))
    else :
        # Exception, la fonction PUPAE n'est pas définie pour i qui n'appartient pas à [|1;LIFESPANpupae|]
        print('Erreur, l age i n est pas possible pour la fonction PUPAE daprès le modele')
        return (-1)

#--------------------------------------------------------------#

# Nombre de cellules contenant des pupes dans la ruche à un instant t 
@lru_cache(maxsize = None)
def CELLSpupae(t):
  # Nombre total de pupes dans la ruche à un instant t  
  return int(sum([PUPAE(i,t) for i in range(1,LIFESPANpupae+1)]))

#--------------------------------------------------------------#

### **CELLULES DU COUVAIN DANS LA RUCHE (2.2)**

In [1377]:
#--------------------------------------------------------------#
@lru_cache(maxsize = None)
def CELLSbrood(t):
  # Nombre d'individus en phase de développement dans la ruche (oeufs+larves+pupes)
  return int(CELLSeggs(t) + CELLSlarvae(t) + CELLSpupae(t))
#--------------------------------------------------------------#  

### **MORTALITE DES ABEILLES ADULTES (2.3)**

In [1378]:
#--------------------------------------------------------------#

# Taux de mortalité des abeilles adultes à un instant t
@lru_cache(maxsize = None)
def MORTALITYadult(t):
    return MORTALITYadultbase + MORTALITYnursing*(NURSES(t-1)/(BEESadult(t-1)+1))+MORTALITYprocessing*(PROCESSORS(t-1)/(BEESadult(t-1)+1))+MORTALITYforaging*(FORAGERSactive(t-1)/(BEESadult(t-1)+1))

#--------------------------------------------------------------#

# Nombre d'abeilles adultes à un instant t
#@lru_cache(maxsize = None)
def BEESadult(t):
    if (t<=0) :
        return BEESadultINIT
    else :
        return int((BEESadult(t-1)+PUPAE(LIFESPANpupae,t-1))*(1-MORTALITYadult(t)))

#--------------------------------------------------------------#

### **INFLUENCE DE LA TEMPERATURE ET DU CLIMAT SUR LE BUTINAGE (2.4)**

In [1379]:
#--------------------------------------------------------------#
def estBissextile(year) :
  bool_biss = False
  if (year%4==0 and year%100!=0 or year%400==0):
    bool_biss = True
  else : 
    print(year, "est une année non bissextile")
  return bool_biss

# Etant donné un jour de l'année (entre 1 et 365) renvoie son mois associé
@lru_cache(maxsize = None)
def mois(t):
  
  #On prend la première ligne des données pour obtenir l'année (ici 2020)
  #On retourne un booleen pour définir si l'année compte 365 ou 366 jours
  var_biss = estBissextile(list_DataMeteo2020[0][0][0])

  if (isinstance(t,int)) :

    if (t>= 1 and t<=31 ):
      # Janvier
      return 1
    
    #L'année est commune et non bissextile
    if (var_biss == False):
      if (t>= 32 and t<=59):
        #Février
        return 2

      if (t>= 60 and t<=90 ):
        # Mars
        return 3  

      if (t>= 91 and t<=120 ):
        # Avril
        return 4 

      if (t>= 121 and t<=151):
        # Mai
        return 5
      
      if (t>= 152 and t<=181):
        # Juin
        return 6

      if (t>= 182 and t<=212):
        # Juillet
        return 7  
      
      if (t>= 213 and t<=243):
        # Août
        return 8    
      
      if (t>= 244 and t<=273):
        # Septembre
        return 9
      
      if (t>= 274 and t<=304):
        # Octobre
        return 10

      if (t>= 305 and t<=334):
        # Novembre
        return 11
          
      if (t>= 335 and t<=365):
        # Décembre
        return 12

    #C'est une année bissextile donc on passe à 366 jour à l'année
    else :
      if (t>= 32 and t<=60):
        #Février
        return 2

      if (t>= 61 and t<=91 ):
        # Mars
        return 3  

      if (t>= 92 and t<=121 ):
        # Avril
        return 4 

      if (t>= 122 and t<=152):
        # Mai
        return 5
      
      if (t>= 153 and t<=182):
        # Juin
        return 6

      if (t>= 183 and t<=213):
        # Juillet
        return 7  
      
      if (t>= 214 and t<=244):
        # Août
        return 8    
      
      if (t>= 245 and t<=274):
        # Septembre
        return 9
      
      if (t>= 275 and t<=305):
        # Octobre
        return 10

      if (t>= 306 and t<=335):
        # Novembre
        return 11
          
      if (t>= 336 and t<=366):
        # Décembre
        return 12
  else :
    # t n'est pas entier ou n'appartient pas à [|1;365|]
    return 99

#--------------------------------------------------------------#  
#Romain 
# Donne la température pour le jour t
#On utilise uniquement les données des témpératures comprises entre 
#L'heure ou le soleil se lève et se couche
@lru_cache(maxsize = None) 
def TEMP(t):
  liste_data = get_dataOfTheDay(t)
  liste_data_temp = []
  #Récupère les data de température uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_temp.append(liste_data[i][2])
  return liste_data_temp
#--------------------------------------------------------------#  
# Donne la durée (en heures) pedant laquelle il pleut lorsqu'il fait jour 
# On utilise le nombre d'heure d'ensoleillement par jour (SUNHOUR)
# D'après l'intensité des précipitations moyennes fournies par Météo France
# http://pluiesextremes.meteo.fr/france-metropole/Intensite-de-precipitations.html
# Et le nombre de millimètre d'eau tombé sur une journée (PRECIP_TOTAL_DAY_MM)
# On peut calculer approximativement le nombre d'heures pendant lesquelles il a plu.
@lru_cache(maxsize = None)
def HOURSraining_during_daylight(t):
  liste_data = get_dataOfTheDay(t)
  liste_data_rain = []
  hours_Day_Light = HOURSdaylight(t)
  #Récupère les data de température uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_rain.append(liste_data[i][3])
  nb_hours_rain = []
  precipitation = liste_data_rain
  for i in range(len(precipitation)):
    value = precipitation[i]
    if (value != 0) : 
      nb_hours_rain.append(value)
  return len(nb_hours_rain)

# Source pluies : https://www.ou-et-quand.net/partir/quand/france/aquitaine/pau-bearn/#:~:text=En%20moyenne%2C%20les%20mois%20les,Janvier%2C%20Avril%2C%20Mai%20et%20Novembre
 
# Donne la durée (en heures) pendant laquelle il fait jour
# On pourrait faire une soustraction des heures de coucher et lever du soleil
@lru_cache(maxsize = None)
def HOURSdaylight(t):
  liste_data = get_dataOfTheDay(t)
  return len(liste_data)

#--------------------------------------------------------------#

# Valentin
# Taux de pluie lorsqu'il fait jour
@lru_cache(maxsize = None)
def RAIN(t):
    if (t<=0):
        return 0
    else:
        if (HOURSdaylight(t)==0):
          return HOURSraining_during_daylight(t)
        else :
          return (HOURSraining_during_daylight(t)/HOURSdaylight(t))

#--------------------------------------------------------------#
# Proche de 1 -> la pluie n'a pas empêché les abeilles d'aller butiner au jour t. 
# Proche de 0 -> la pluie a empêché les abeilles d'aller butiner pendant la quasi totalité de la journée.
@lru_cache(maxsize = None)
def INDEXrain(t):
    if ((1-RAIN(t)) < 0) :
      return 0
    else :
      return 1-RAIN(t)

#--------------------------------------------------------------#  Facteur saison
# Romain
# Proche de 1 -> la température était favorable au butinage au jour t. 
# Proche de 0 -> la température a empêché les abeilles d'aller butiner pendant la quasi totalité de la journée.
@lru_cache(maxsize = None)
def INDEXtemperature(t):
    if (t<= 0):
        return 0
    else:
        temp = TEMP(t)
        index_res = []
        for i in range(len(temp)):
          value = temp[i]
          if (value <= 14):
            index_res.append(0)
          if (value>14 and value<=22):
            res = (value - 14)/8
            index_res.append(res)
          if (value>22 and value<=32):
            index_res.append(1)
          if (value>32 and value<=40):
            res = (40-value)/8
            index_res.append(res)
          if (value>40):
            index_res.append(0)
    if len(temp)==0:
      return 0
    else :
      return sum(index_res)/len(temp)
#--------------------------------------------------------------# 
# Guillaume
# Index du vent en fonction de sa vitesse
# 0 les abeilles ne sortent pas (vent > 45 km/h)
# 1 les abeilles sortent (vent < 45km/h)
def INDEXwind(t):
  liste_data = get_dataOfTheDay(t)
  liste_data_wind = []
  #Récupère les data de vent uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_wind.append(liste_data[i][5])

  wind = liste_data_wind
  index_res = []
  for i in range(len(wind)):
    value = wind[i]
    if (value>=0 and value<=max_wind):
      index_res.append(1-(value/max_wind))
    else :
      index_res.append(0)
  if len(wind) == 0:
    return 0
  else :
    return sum(index_res)/len(wind)
#--------------------------------------------------------------#  
# Mathis
# Index de l'humidité en %
# 0 les abeilles ne sortent pas (humidity < 60 )
# 1 les abeilles sortent (humidity > 60)

# Selon l'article suivant, les abeilles vont prioriser la récolte de nectar
# à celle du pollen dans le cas d'une forte humidité (ou lorsqu'il vient de 
# pleuvoir)
# https://www.jstor.org/stable/25063598?seq=1

# De plus un trop haut taux d'humidity affecte négativement la qualité 
# du pollen et le nectar 
# Silva et al., 2013; Alves et al., 2015
def INDEXhumidity(t):
  print("Jour=",t)
  liste_data = get_dataOfTheDay(t)
  liste_data_humidity = []
  #Récupère les data d'humidité uniquement et retourne leur liste
  for i in range(len(liste_data)):
    liste_data_humidity.append(liste_data[i][4])
    
  humidity = liste_data_humidity
  index_res = []
  print("HUMIDITY=",humidity)
  for i in range(len(humidity)):
    value = humidity[i]
    if (value>=min_humidity and value<=80):
      index_res.append(1)
    if (value>80 and value<=100):
      index_res.append((100-value)/20)
    if (value<min_humidity):
       index_res.append(0)
  print(index_res)
  if len(humidity) == 0:
    return 0
  else :
    print("RES= ",sum(index_res)/len(humidity))
    return sum(index_res)/len(humidity)
#--------------------------------------------------------------# 

# Proche de 1 -> Les conditions climatiques étaient favorables au butinage. 
# Proche de 0 -> Les conditions climatiques ont empêché les abeilles d'aller butiner pendant la quasi totalité de la journée.
@lru_cache(maxsize = None)
def INDEXflight(t):
    return INDEXrain(t) * INDEXtemperature(t) * INDEXwind(t) #* INDEXhumidity(t)

#--------------------------------------------------------------# 
@lru_cache(maxsize = None)
def INDEXnectaroutside(t):
    return min((1-season(t))*1.5,1)

#--------------------------------------------------------------# 
@lru_cache(maxsize = None)
def INDEXpollenoutside(t):
    return min((1-season(t))*1.5,1)

#--------------------------------------------------------------#

### **DECISION ET REGULATION DES TACHES DE TRAVAIL (2.5 et 2.6)**

In [1380]:
#--------------------------------------------------------------#

# Modelise le besoin journalier en abeilles travailleuses
@lru_cache(maxsize = None)
def NEEDworkers(t):
    return NEEDnurses(t)+NEEDpollenforagers(t)

#--------------------------------------------------------------#

# calcul le ratio des abeilles disponibles pour le travail
# si ratio < 1 alors (1-RATIOworkforce(t)) du travail necessaire ne sera pas effectué
@lru_cache(maxsize = None)
def RATIOworkforce(t):
    ratio=(BEESadult(t)*(1-FACTORothertasks))/(NEEDworkers(t)+1)
    if ratio<1:
        return ratio
    else:
        return 1

#--------------------------------------------------------------#

# Somme toute les demandes nouricieres de chaque couvins
@lru_cache(maxsize = None)
def NEEDnurses(t):
    total=0
    for i in range(1,LIFESPANlarvae+1):
        total+=LARVAE(i,t)*NEEDnurses_per_larva[i-1]
    return int(total+(CELLSeggs(t)*NEEDnurses_per_egg)+(CELLSpupae(t)*NEEDnurses_per_pupa))

#--------------------------------------------------------------#

# Nombre d'abeilles nouricieres actives
@lru_cache(maxsize = None)
def NURSES(t):
    return (NEEDnurses(t)*RATIOworkforce(t))

#--------------------------------------------------------------#

# Permet de modeliser une importante boucle de retour
@lru_cache(maxsize = None)
def INDEXnursingquality(t):
    return NURSES(t)/(NEEDnurses(t)+1)

#--------------------------------------------------------------#

### **BUTINAGE ET BESOINS EN RESSOURCES (2.7)**

In [1381]:
#--------------------------------------------------------------#

# Calcule le nombre de butineuses potentielles
@lru_cache(maxsize = None)
def FORAGERS(t):
	return int(FORAGERSpollen(t) + FORAGERSnectar(t))

#--------------------------------------------------------------#

# Calcule le nombre de butineuses qui sortent de la ruche
@lru_cache(maxsize = None)
def FORAGERSactive(t):
	return (FORAGERSpollenactive(t) + FORAGERSnectaractive(t))

#--------------------------------------------------------------#

# Nombre d'abeilles butineuses actives
@lru_cache(maxsize = None)
def FORAGERSnectaractive(t):
    return FORAGERSnectar(t)*INDEXflight(t)*INDEXnectaroutside(t)

#--------------------------------------------------------------#

# Nombre d'abeilles disponibles pour aller recolter du nectar
@lru_cache(maxsize = None)
def FORAGERSnectar(t):
    return min(BEESadult(t)*FACTORforagingmax-FORAGERSpollen(t),WORKFORCEnectar(t)-PROCESSORS(t))

#--------------------------------------------------------------#

# Besoin en pollen total
@lru_cache(maxsize = None)
def NEEDpollen(t):
    if (t<=0):
        return 0
    else:
        return NEEDpollen_larvae(t)+NEEDpollen_adult(t)

#--------------------------------------------------------------#

# Calcule le besoin en pollen de la part des larves
@lru_cache(maxsize = None)
def NEEDpollen_larvae(t):
    res = 0
    for i in range(1,LIFESPANlarvae+1):
        res = res + POLLENNEEDlarva[i-1]*LARVAE(i,t)
    return res

#--------------------------------------------------------------#

# Calcule le besoin en pollen de la part des abeilles adultes
@lru_cache(maxsize = None)
def NEEDpollen_adult(t):
    return BEESadult(t)*POLLENNEEDadult + NURSES(t)*POLLENNEEDnurse

#--------------------------------------------------------------#

# Calcule le besoin journalier en pollen à recolter basé sur les reserves moyenées entre j0 et j-2
@lru_cache(maxsize = None)
def NEEDpollenincome(t):
    acc = 0
    for d in range(3):
        acc = acc + NEEDpollen(t-d)
    acc = acc/3 * FACTORpollenstorage - STORESpollen(t)
    return max(0, acc)

#--------------------------------------------------------------#

# Calcule le nombre de butineuses de pollen requis
@lru_cache(maxsize = None)
def NEEDpollenforagers(t):
    return (NEEDpollenincome(t-1) / (LOADpollenforager*TURNSpollenforager*FACTORforagingsuccess))

#--------------------------------------------------------------#

# Calcule le nombre potentiel de butineuses de pollen
@lru_cache(maxsize = None)
def FORAGERSpollen(t):
    maximum = max(NEEDpollenforagers(t)*RATIOworkforce(t), (BEESadult(t)-NURSES(t))*FACTORminpollenforagers )
    return min(maximum, BEESadult(t)*FACTORforagingmax)

#--------------------------------------------------------------#

# Calcule le nombre de butineuses de pollen qui sortent de la ruche
@lru_cache(maxsize = None)
def FORAGERSpollenactive(t):
    return FORAGERSpollen(t)*INDEXflight(t)*INDEXpollenoutside(t)
 

#--------------------------------------------------------------#

# Calcule le besoin journalier en nectar de la ruche
@lru_cache(maxsize = None)
def NEEDnectar(t):
    return NEEDnectar_larvae(t)+NEEDnectar_adult(t)
#--------------------------------------------------------------#

# Calcule le besoin en nectar de la part des larves
@lru_cache(maxsize = None)
def NEEDnectar_larvae(t):
    acc = 0
    for i in range(1,LIFESPANlarvae+1):
        acc = acc + (NECTARNEEDlarva[i-1]*LARVAE(i,t))
    return acc

#--------------------------------------------------------------#

# Calcule la demande en nectar de la part des abeilles adultes
@lru_cache(maxsize = None)
def NEEDnectar_adult(t):
    # nectar necessaire pour les abeilles adultes
    nectar_adults = BEESadult(t)*NECTARNEEDadult

    # nectar necessaire pour les abeilles nouricieres
    nectar_nurses = NURSES(t)*NECTARNEEDnurse

    # nectar necessaire pour les butineuses actives
    nectar_foragers = FORAGERSactive(t)*NECTARNEEDactiveforager

    # Retourne la demande en nectar des abeilles adultes
    return nectar_adults+nectar_nurses+nectar_foragers

#--------------------------------------------------------------#

# Calcule la force de travail restante pour le nectar
@lru_cache(maxsize = None)
def WORKFORCEnectar(t):
    if RATIOworkforce(t)==1:
        res = BEESadult(t)*(1-FACTORothertasks)
        return res-NURSES(t)-FORAGERSpollen(t)
    else :
        return 0

#--------------------------------------------------------------#

### **FLUX DE RESSOURCES ET TRAITEMENT DE LA NOURRITURE AU SEIN DE LA COLONIE (2.8  et 2.9)**

In [1382]:
#--------------------------------------------------------------#

# Permet de calculer le pollen produit chaque jour
@lru_cache(maxsize = None)
def INCOMEpollen(t):
    return FORAGERSpollenactive(t) * LOADpollenforager * TURNSpollenforager * FACTORforagingstoch(t) * FACTORforagingsuccess

#--------------------------------------------------------------#

# Permet de calculer le facteur stochastique
@lru_cache(maxsize = None)
def FACTORforagingstoch(t):
    return(rd.uniform(1-stochasticForagingFactor , 1+stochasticForagingFactor))

#--------------------------------------------------------------#

# Decrit le niveau de pollen stocké par rapport aux besoins de la population
@lru_cache(maxsize = None)
def INDEXpollensituation(t):
    return min([1 , (STORESpollen(t)) / (NEEDpollen(t)*FACTORpollenstorage+1)])

#--------------------------------------------------------------#

# Décrit la production de nectar
@lru_cache(maxsize = None)
def INCOMEnectar(t):
    return (min([FORAGERSnectaractive(t) * LOADnectarforager * TURNSnectarforager * FACTORforagingstoch(t) * FACTORforagingsuccess , CELLSempty(t-1)]))

#--------------------------------------------------------------#

# Determine les processeurs de nectar necessaires
@lru_cache(maxsize = None)
def NEEDprocessors(t):
    return STORESnectar(t-1) * ProcessorsPerCell

#--------------------------------------------------------------#

@lru_cache(maxsize = None)
def PROCESSORS(t):
    return (min([NEEDprocessors(t) , WORKFORCEnectar(t)]))

#--------------------------------------------------------------#

# Determine le nectar produit 
@lru_cache(maxsize = None)
def PROCESSEDnectar(t):
    return (min([STORESnectar(t-1) - USAGEnectar(t) , PROCESSORS(t) / ProcessorsPerCell]))

#--------------------------------------------------------------#

### **GESTION DES RESSOURCES ET TRAITEMENT (2.10)**

In [1383]:
#--------------------------------------------------------------#

# Modelise l'utilisation du nectar dans la colonie
@lru_cache(maxsize = None)
def USAGEnectar(t):
    liste_var = []
    needNectar_var = NEEDnectar(t)
    storeNectar_var = STORESnectar(t-1)
    liste_var.append(needNectar_var)
    liste_var.append(storeNectar_var)
    return min(liste_var)

#--------------------------------------------------------------#

#Modelise l'utilisation du pollen dans la colonie
@lru_cache(maxsize = None)
def USAGEpollen(t):
    liste_var = []
    needPollen_var = NEEDpollen(t-1)*(1-(FACTORpollensavingmax*(1-INDEXpollensituation(t-1))))
    storePollen_var = STORESpollen(t-1)

    liste_var.append(needPollen_var)
    liste_var.append(storePollen_var)
    return min(liste_var)

#--------------------------------------------------------------#

# Modelise l'utilisation du miel dans la colonie
@lru_cache(maxsize = None)
def USAGEhoney(t):
    liste_var = []
    storeHoney_var = STOREShoney(t-1)
    nectar_var = (NEEDnectar(t)-USAGEnectar(t))*RATIOnectar_to_honey
    liste_var.append(storeHoney_var)
    liste_var.append(nectar_var)

    return min(liste_var)


#--------------------------------------------------------------#

# Modelise le stockage du pollen dans la colonie
@lru_cache(maxsize = None)
def STORESpollen(t):
    if (t<=0):
        return 0
    else:
        return STORESpollen(t-1)+INCOMEpollen(t)-USAGEpollen(t)

#--------------------------------------------------------------#

# Modelise le stockage du nectar dans la colonie
@lru_cache(maxsize = None)
def STORESnectar(t):
    if (t<=0):
        return 0
    else :
        return STORESnectar(t-1)+INCOMEnectar(t)-USAGEnectar(t)-PROCESSEDnectar(t)

#--------------------------------------------------------------#

# Modelise le stockage du miel dans la colonie
@lru_cache(maxsize = None)
def STOREShoney(t):
    if (t<=0):
        return STOREShoneyINIT
    else :  
        return STOREShoney(t-1)-USAGEhoney(t)+(PROCESSEDnectar(t)*RATIOnectar_to_honey)

#--------------------------------------------------------------#

# Modelise le poids de la colonie
@lru_cache(maxsize = None)
def WEIGHTcolony(t):
    somme_res = 0
    for i in range (1, LIFESPANlarvae):
        somme_res += w_larva[i-1]*LARVAE(i,t)
        return 1/1000 * (w_hivebase + (w_cellsbase * CELLShive) + (w_pollen * STORESpollen(t)) + (w_nectar * STORESnectar(t)) + (w_honey * STOREShoney(t)) + (w_egg * CELLSeggs(t)) + (w_pupa * CELLSpupae(t)) + somme_res + w_adult * BEESadult(t) )

#--------------------------------------------------------------#

# Modelise le nombre d'abeilles paresseuses dans la colonie
@lru_cache(maxsize = None)
def BEESlazy(t):
    return (BEESadult(t)*(1-FACTORothertasks))-FORAGERSactive(t)-NURSES(t)-PROCESSORS(t)

#--------------------------------------------------------------#

### **TEST DES FONCTIONS**

In [1384]:
BEESadult(365)

10730

# PARTIE LPO

In [1385]:
def HOPOMO(nbdays):
    beesadult = []
    beeslazy = []
    cellsbrood = []
    cellseggs = []
    cellsempty = []
    cellslarvae = []
    cellspupae = []
    elr = []
    foragers = []
    foragersactive = []
    foragersnectar = []
    foragersnectaractive = []
    foragersnectaractiveTEST = []
    foragerspollen = []
    foragerspollenactive = []
    incomenectar = []
    incomepollen = []
    indexflight = []
    indexrain = []
    indexvent = []
    indexhumidity = []
    indextemperature = []
    indexnectaroutside = []
    indexnursingquality = []
    indexpollenoutside = []
    indexpollensituation = []
    larvae1 = []
    larvae2 = []
    larvae3 = []
    larvae4 = []
    larvae5 = []
    mortalityadult = []
    neednectar = []
    neednectar_adult = []
    neednectar_larvae = []
    neednurses = []
    needpollen = []
    needpollen_adult = []
    needpollen_larvae = []
    needpollenforagers = []
    needpollenincome = []
    needprocessors = []
    needworkers = []
    nurses = []
    processednectar = []
    processors = []
    ratioworkforce = []
    storeshoney = []
    storesnectar = []
    storespollen = []
    supcomb = []
    #temp = np.concatenate([temp1,temp2])
    #temp1 = np.linspace(15, 35, 183)
    #temp2 = np.linspace(35, 15, 183)
    usagehoney = []
    usagenectar = []
    usagepollen = []
    weightcolony = []
    workforcenectar = []

    t = np.arange(nbdays+1)
    for i in range(len(t)):
        beesadult.append(BEESadult(i))
        beeslazy.append(BEESlazy(i))
        cellsbrood.append(CELLSbrood(i))
        cellseggs.append(CELLSeggs(i))
        cellsempty.append(CELLSempty(i))
        cellslarvae.append(CELLSlarvae(i))
        cellspupae.append(CELLSpupae(i))
        elr.append(ELR(i))
        foragers.append(FORAGERS(i))
        foragersactive.append(FORAGERSactive(i))
        foragersnectar.append(FORAGERSnectar(i))
        foragersnectaractive.append(FORAGERSnectaractive(i))
        foragerspollen.append(FORAGERSpollen(i))
        foragerspollenactive.append(FORAGERSpollenactive(i))
        incomenectar.append(INCOMEnectar(i))
        incomepollen.append(INCOMEpollen(i))
        indexflight.append(INDEXflight(i))        
        indexrain.append(INDEXrain(i))
        indexvent.append(INDEXwind(i))
        indexhumidity.append(INDEXhumidity(i))
        indextemperature.append(INDEXtemperature(i))
        indexnectaroutside.append(INDEXnectaroutside(i))
        indexnursingquality.append(INDEXnursingquality(i))
        indexpollenoutside.append(INDEXpollenoutside(i))
        indexpollensituation.append(INDEXpollensituation(i))
        larvae1.append(LARVAE(1,i))
        larvae2.append(LARVAE(2,i))
        larvae3.append(LARVAE(3,i))
        larvae4.append(LARVAE(4,i))
        larvae5.append(LARVAE(5,i))
        mortalityadult.append(MORTALITYadult(i))
        neednectar_adult.append(NEEDnectar_adult(i))
        neednectar_larvae.append(NEEDnectar_larvae(i))
        neednectar.append(NEEDnectar(i))
        neednurses.append(NEEDnurses(i))
        needpollen_adult.append(NEEDpollen_adult(i))
        needpollen_larvae.append(NEEDpollen_larvae(i))
        needpollen.append(NEEDpollen(i))
        needpollenforagers.append(NEEDpollenforagers(i))
        needpollenincome.append(NEEDpollenincome(i))
        needprocessors.append(NEEDprocessors(i))
        needworkers.append(NEEDworkers(i))
        nurses.append(NURSES(i))
        processednectar.append(PROCESSEDnectar(i))
        processors.append(PROCESSORS(i))
        ratioworkforce.append(RATIOworkforce(i))
        storeshoney.append(STOREShoney(i))
        storesnectar.append(STORESnectar(i))
        storespollen.append(STORESpollen(i))
        supcomb.append(SUPcomb(i))
        usagehoney.append(USAGEhoney(i))
        usagenectar.append(USAGEnectar(i))
        usagepollen.append(USAGEpollen(i))
        weightcolony.append(WEIGHTcolony(i))
        workforcenectar.append(WORKFORCEnectar(i))
        
    hopomo = pd.DataFrame({
        'beesadult' : beesadult,
        'beeslazy' : beeslazy,
        'cellsbrood' : cellsbrood,
        'cellseggs' : cellseggs,
        'cellsempty' : cellsempty,
        'cellslarvae' : cellslarvae,
        'cellspupae' : cellspupae,
        'elr' : elr,
        'foragers' : foragers,
        'foragersactive' : foragersactive,
        'foragersnectar' : foragersnectar,
        'foragersnectaractive' : foragersnectaractive,
        'foragerspollen' : foragerspollen,
        'foragerspollenactive' : foragerspollenactive,
        'incomenectar' : incomenectar,
        'incomepollen' : incomepollen,
        'indexflight' : indexflight,        
        'indexrain' : indexrain,
        'indexvent' : indexvent, 
        'indexhumidity' : indexhumidity,       
        'indextemperature' : indextemperature,
        'indexnectaroutside' : indexnectaroutside,
        'indexnursingquality' : indexnursingquality,
        'indexpollenoutside' : indexpollenoutside,
        'indexpollensituation' : indexpollensituation,
        'larvae1' : larvae1,
        'larvae2' : larvae2,
        'larvae3' : larvae3,
        'larvae4' : larvae4,
        'larvae5' : larvae5,
        'mortalityadult' : mortalityadult,
        'neednectar_adult' : neednectar_adult,
        'neednectar_larvae' : neednectar_larvae,
        'neednectar' : neednectar,
        'neednurses' : neednurses,
        'needpollen_adult' : needpollen_adult,
        'needpollen_larvae' : needpollen_larvae,
        'needpollen' : needpollen,
        'needpollenforagers' : needpollenforagers,
        'needpollenincome' : needpollenincome,
        'needprocessors' : needprocessors,
        'needworkers' : needworkers,
        'nurses' : nurses,
        'processednectar' : processednectar,
        'processors' : processors,
        'ratioworkforce' : ratioworkforce,
        'storeshoney' : storeshoney,
        'storesnectar' : storesnectar,
        'storespollen' : storespollen,
        'supcomb' : supcomb,
        'usagehoney' : usagehoney,
        'usagenectar' : usagenectar,
        'usagepollen' : usagepollen,
        'weightcolony' : weightcolony,
        'workforcenectar' : workforcenectar,
                                })
    return hopomo

In [1386]:
hopomo = HOPOMO(364)

Jour= 0
HUMIDITY= []
[]
Jour= 1
HUMIDITY= [81, 70, 71, 62, 62, 62, 62, 66, 81]
[0.95, 1, 1, 1, 1, 1, 1, 1, 0.95]
RES=  0.9888888888888889
Jour= 2
HUMIDITY= [86, 80, 65, 58, 54, 62, 67, 71, 71, 81]
[0.7, 1, 1, 0, 0, 1, 1, 1, 1, 0.95]
RES=  0.765
Jour= 3
HUMIDITY= [93, 93, 100, 81, 76, 71, 66, 71, 76, 93]
[0.35, 0.35, 0.0, 0.95, 1, 1, 1, 1, 1, 0.35]
RES=  0.7
Jour= 4
HUMIDITY= [100, 100, 100, 100, 87, 81, 87, 76, 87, 81]
[0.0, 0.0, 0.0, 0.0, 0.65, 0.95, 0.65, 1, 0.65, 0.95]
RES=  0.485
Jour= 5
HUMIDITY= [100, 100, 100, 100, 100, 100, 100, 93, 100, 93]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.0, 0.35]
RES=  0.06999999999999999
Jour= 6
HUMIDITY= [100, 100, 100, 100, 100, 93, 93, 93, 87, 93]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.35, 0.35, 0.65, 0.35]
RES=  0.205
Jour= 7
HUMIDITY= [81, 81, 76, 62, 66, 62, 58, 58, 66, 71]
[0.95, 0.95, 1, 1, 1, 1, 0, 0, 1, 1]
RES=  0.79
Jour= 8
HUMIDITY= [93, 87, 81, 71, 58, 54, 55, 59, 62, 76]
[0.35, 0.65, 0.95, 1, 0, 0, 0, 0, 1, 1]
RES=  0.495
Jour= 9
HUMIDIT

In [1387]:
hopomo.to_csv("hopomo.csv")
hopomo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   beesadult             365 non-null    int64  
 1   beeslazy              365 non-null    float64
 2   cellsbrood            365 non-null    int64  
 3   cellseggs             365 non-null    int64  
 4   cellsempty            365 non-null    int64  
 5   cellslarvae           365 non-null    int64  
 6   cellspupae            365 non-null    int64  
 7   elr                   365 non-null    float64
 8   foragers              365 non-null    int64  
 9   foragersactive        365 non-null    float64
 10  foragersnectar        365 non-null    float64
 11  foragersnectaractive  365 non-null    float64
 12  foragerspollen        365 non-null    float64
 13  foragerspollenactive  365 non-null    float64
 14  incomenectar          365 non-null    float64
 15  incomepollen          3

In [1388]:
fig = make_subplots(rows=3, cols=2, vertical_spacing=0.05)

fig.add_trace(
    go.Scatter(y=hopomo.beesadult, name="Adult Bees", mode='lines', xaxis="x", yaxis="y1", showlegend = True),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=hopomo.storespollen, name="Pollen Stores", mode='lines', xaxis="x", yaxis="y2",showlegend = True),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(y=hopomo.cellsbrood, name="Brood cells", mode='lines', xaxis="x", yaxis="y3",showlegend = True),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(y=hopomo.storeshoney, name="Honey Stores", mode='lines', xaxis="x", yaxis="y4", showlegend = True),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(y=hopomo.weightcolony, name="Colony Weight", mode='lines', xaxis="x", yaxis="y6",showlegend = True),
    row=3, col=2
)

fig.update_yaxes(title_text="adult bees", range=[0, 60000], row=1, col=1)
fig.update_yaxes(title_text="pollen stores", range=[0,6000], row=1, col=2)
fig.update_yaxes(title_text="brood cells", range=[0, 35000], row=2, col=1)
fig.update_yaxes(title_text="honey stores", range=[0, 180000],nticks=10, row=2, col=2)
fig.update_yaxes(title_text="colony weight (kg)", range=[0, 180],nticks=10, row=3, col=2)
#fig.update_yaxes(title_text = 'Temp (°C)', ticks="outside", range=[-4, 30], tickwidth=1, nticks=30, tickcolor='crimson', showgrid=False, showline = True,secondary_y=False, spikesnap='cursor',spikethickness=1)
fig.update_traces(hovertemplate="day: %{x} <br>val: %{y:.1f}")
fig.update_layout(autosize=False, height=1200, width=1200)
fig.show()

## tous les params

In [1389]:
fig2 = make_subplots(rows=10, cols=1, vertical_spacing=0.005, shared_xaxes=True)
# Cells
fig2.add_trace(go.Scatter(y=hopomo.storeshoney, name="storeshoney", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.storesnectar, name="storesnectar", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.storespollen, name="storespollen", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1) 
fig2.add_trace(go.Scatter(y=hopomo.cellseggs, name="cellseggs", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellslarvae, name="cellslarvae", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellspupae, name="cellspupae", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellsbrood, name="cellsbrood", mode='lines', xaxis="x", yaxis="y1"),
    row=1, col=1)
fig2.add_trace(go.Scatter(y=hopomo.cellsempty, name="cellsempty", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=1, col=1)


fig2.add_trace(go.Scatter(y=hopomo.incomenectar, name="incomenectar", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=2, col=1)
fig2.add_trace(go.Scatter(y=hopomo.incomepollen, name="incomepollen", mode='lines', xaxis="x", yaxis="y1",stackgroup='one'),
    row=2, col=1)

#Indexes
fig2.add_trace(go.Scatter(y=hopomo.indexflight, name="indexflight", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexrain, name="indexrain", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indextemperature, name="index temperature", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexvent, name="indexvent", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexhumidity, name="indexhumidity", mode='lines', xaxis="x", yaxis="y1"),
    row=3, col=1)

fig2.add_trace(go.Scatter(y=hopomo.indexnectaroutside, name="index nectaroutside", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexpollenoutside, name="index pollenoutside", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexpollensituation, name="index pollensituation", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)
fig2.add_trace(go.Scatter(y=hopomo.indexnursingquality, name="index nursingquality", mode='lines', xaxis="x", yaxis="y1"),
    row=4, col=1)

fig2.add_trace(go.Scatter(y=hopomo.foragersactive, name="foragersactive", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragersnectar, name="foragersnectar", mode='lines', xaxis="x", yaxis="y1"),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragersnectaractive, name="foragersnectar active", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragerspollen, name="foragerspollen", mode='lines', xaxis="x", yaxis="y1"),
    row=5, col=1)
fig2.add_trace(go.Scatter(y=hopomo.foragerspollenactive, name="foragerspollen active", mode='lines', xaxis="x", yaxis="y1",fill='tozeroy'),
    row=5, col=1)

fig2.add_trace(go.Scatter(y=hopomo.nurses, name="nurses", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.neednurses, name="neednurses", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needworkers, name="needworkers", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needprocessors, name="needprocessors", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollenforagers, name="needpollenforagers", mode='lines', xaxis="x", yaxis="y1"),
    row=6, col=1)

fig2.add_trace(go.Scatter(y=hopomo.larvae1, name="larvae 1", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae2, name="larvae 2", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae3, name="larvae 3", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae4, name="larvae 4", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)
fig2.add_trace(go.Scatter(y=hopomo.larvae5, name="larvae 5", mode='lines', xaxis="x", yaxis="y1"),
    row=7, col=1)

fig2.add_trace(go.Scatter(y=hopomo.needpollen, name="needpollen", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollenincome, name="needpollen income", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollen_adult, name="needpollen_adult", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.needpollen_larvae, name="needpollen_larvae", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)
fig2.add_trace(go.Scatter(y=hopomo.storespollen, name="storespollen", mode='lines', xaxis="x", yaxis="y1"),
    row=8, col=1)


fig2.add_trace(go.Scatter(y=hopomo.neednectar, name="neednectar", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)
fig2.add_trace(go.Scatter(y=hopomo.neednectar_adult, name="neednectar_adult", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)
fig2.add_trace(go.Scatter(y=hopomo.neednectar_larvae, name="neednectar_larvae", mode='lines', xaxis="x", yaxis="y1"),
    row=9, col=1)



fig2.add_trace(go.Scatter(y=hopomo.usagehoney, name="usagehoney", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)
fig2.add_trace(go.Scatter(y=hopomo.usagenectar, name="usagenectar", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)
fig2.add_trace(go.Scatter(y=hopomo.usagepollen, name="usagepollen", mode='lines', xaxis="x", yaxis="y1"),
    row=10, col=1)


fig2.update_yaxes(title_text="cells", range=[0, 110000], row=1, col=1)
fig2.update_yaxes(title_text="incomes", row=2, col=1)
fig2.update_yaxes(title_text="indexes weather", nticks=10, row=3, col=1)
fig2.update_yaxes(title_text="indexes", nticks=10, row=4, col=1)
fig2.update_yaxes(title_text="foragers", nticks=10, row=5, col=1)
fig2.update_yaxes(title_text="need bees", nticks=10, row=6, col=1)
fig2.update_yaxes(title_text="larvae 1-5", nticks=10, row=7, col=1)
fig2.update_yaxes(title_text="need pollen", nticks=10, row=8, col=1)
fig2.update_yaxes(title_text="need nectar", nticks=10, row=9, col=1)
fig2.update_yaxes(title_text="usage poll & nectar", nticks=10, row=10, col=1)

fig.update_xaxes(title_text = 'Temp (°C)', ticks="outside", nticks=12, tickcolor='crimson', showline = True)

fig2.update_traces(hovertemplate="day: %{x} <br>val: %{y:.1f}")
fig2.update_layout(autosize=False, height=3000, width=1200)
fig2.show()